In [14]:
#%pip install -e ../.
%pip uninstall -y polynx 
#%pip install update polars

Found existing installation: polynx 0.1.0
Uninstalling polynx-0.1.0:
  Successfully uninstalled polynx-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
sys.path.insert(0, "../src")
import polynx
import gc
from polynx import DataFrame, LazyFrame
import polars as pl
import polynx as plx
gc.collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


3624

In [6]:
df = pl.DataFrame({
    'A':[1, 2, 3, 4], 
    'B': ['abc', 'bc', 'aaa', None], 
    'C': ['2023-01-01','2021-01-01','2009-11-01','2000-11-11'],
    'E': [1.1, 2.1, 3.5, 0]}
).with_columns(pl.col('C').str.to_datetime(format='%Y-%m-%d',time_unit='ns').dt.date())

df_plx = DataFrame(df)

In [8]:
#len(dir(plx)), len(dir(pl))
dir(df_plx)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_wrapped',
 '_pl',
 'asort',
 'assign',
 'cum_max',
 'dd',
 'describe',
 'dsort',
 'eval',
 'gb',
 'max',
 'min',
 'pplot',
 'query',
 'round',
 'to_list',
 'to_polars',
 'ucnt',
 'unstack',
 'vcnt',
 'wc']

In [39]:
#dir(df_plx.lazy())
df_plx.wc("E = 1")
#type(df_plx)

A,B,C,E
i64,str,date,i32
1,"""abc""",2023-01-01,1
2,"""bc""",2021-01-01,1
3,"""aaa""",2009-11-01,1
4,null,2000-11-11,1


In [12]:
df_plx.lazy().describe()

statistic,A,B,C,E
str,f64,str,str,f64
"""count""",4.0,"""3""","""4""",4.0
"""null_count""",0.0,"""1""","""0""",0.0
"""mean""",2.5,null,"""2013-09-03 18:00:00""",1.675
"""std""",1.290994,null,null,1.488568
"""min""",1.0,"""aaa""","""2000-11-11""",0.0
"""25%""",2.0,null,"""2009-11-01""",1.1
"""50%""",3.0,null,"""2021-01-01""",2.1
"""75%""",3.0,null,"""2021-01-01""",2.1
"""max""",4.0,"""bc""","""2023-01-01""",3.5


In [12]:
#plx.DataFrame()
df_plx.describe(group_keys='B', selected_columns=['A','E']).round()

B,A_count,A_nulls,A_mean,A_std,A_min,A_25%,A_50%,A_75%,A_max,E_count,E_nulls,E_mean,E_std,E_min,E_25%,E_50%,E_75%,E_max
str,u32,u32,f64,f64,i64,f64,f64,f64,i64,u32,u32,f64,f64,f64,f64,f64,f64,f64
"""abc""",1,0,1.0,null,1,1.0,1.0,1.0,1,1,0,1.1,null,1.1,1.1,1.1,1.1,1.1
"""bc""",1,0,2.0,null,2,2.0,2.0,2.0,2,1,0,2.1,null,2.1,2.1,2.1,2.1,2.1
null,1,0,4.0,null,4,4.0,4.0,4.0,4,1,0,0.0,null,0.0,0.0,0.0,0.0,0.0
"""aaa""",1,0,3.0,null,3,3.0,3.0,3.0,3,1,0,3.5,null,3.5,3.5,3.5,3.5,3.5


In [22]:
var5 ='2001-01-01'
var3 = '2020-01-01'
query_str =  "@var1 <= A < @var2 & C.dt.year >=2020 & B in @var4"
query_str = "B.str.contains('a|b')"
query_str = "C >'2010-01-01' & A > 1"
#query_str = " B in ['aaa','bc']"
#query_str = " A == 1"
#query_str = " A ** 2 - E > 1"
#query_str = " @var5 < C < @var3 "
#query_str = " B.str.ends_with('c')"
#query_str = " not (B.is_null() & A.is_not_null())"
#query_str = " A ** 2 - E > 1"
query_str = "A.sum().over('B') > 1"

df_plx.query(query_str)

#dir(df)


A,B,C,E
i64,str,datetime[ns],f64
2,"""bc""",2021-01-01 00:00:00,2.1
3,"""aaa""",2009-11-01 00:00:00,3.5
4,null,2000-11-11 00:00:00,0.0
